In [407]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

from IPython.display import Image
import graphviz
import pydotplus 

pd.options.display.max_columns = 25

In [2]:
df_austin = pd.read_csv("austin_weather.csv")
df_austin['Events'].replace(' ', np.nan, inplace=True)
df_austin['Events'].replace(' ', np.nan, inplace=True)
df_austin['Events'].fillna('clear', inplace=True)

In [17]:
df_austin.replace('-', 0, inplace=True)

In [206]:
df_austin['PrecipitationSumInches'].replace('T', 0, inplace=True)

In [207]:
df_austin.head()

,Date,TempHighF,TempAvgF,TempLowF,DewPointHighF,DewPointAvgF,DewPointLowF,HumidityHighPercent,HumidityAvgPercent,HumidityLowPercent,SeaLevelPressureHighInches,SeaLevelPressureAvgInches,SeaLevelPressureLowInches,VisibilityHighMiles,VisibilityAvgMiles,VisibilityLowMiles,WindHighMPH,WindAvgMPH,WindGustMPH,PrecipitationSumInches,Events
0,2013-12-21,74,60,45,67,49,43,93,75,57,29.86,29.68,29.59,10,7,2,20,4,31,0.46,"Rain , Thunderstorm"
1,2013-12-22,56,48,39,43,36,28,93,68,43,30.41,30.13,29.87,10,10,5,16,6,25,0,clear
2,2013-12-23,58,45,32,31,27,23,76,52,27,30.56,30.49,30.41,10,10,10,8,3,12,0,clear
3,2013-12-24,61,46,31,36,28,21,89,56,22,30.56,30.45,30.3,10,10,7,12,4,20,0,clear
4,2013-12-25,58,50,41,44,40,36,86,71,56,30.41,30.33,30.27,10,10,7,10,2,16,0,clear


In [208]:
no_use = ["Date", "Events"]
target = "Events"
features = [i for i in df_austin if i not in no_use]

In [239]:
df_austin.corr()

,TempHighF,TempAvgF,TempLowF
TempHighF,1.000000,0.970655,0.881977
TempAvgF,0.970655,1.000000,0.968573
TempLowF,0.881977,0.968573,1.000000


In [209]:
features

['TempHighF',
 'TempAvgF',
 'TempLowF',
 'DewPointHighF',
 'DewPointAvgF',
 'DewPointLowF',
 'HumidityHighPercent',
 'HumidityAvgPercent',
 'HumidityLowPercent',
 'SeaLevelPressureHighInches',
 'SeaLevelPressureAvgInches',
 'SeaLevelPressureLowInches',
 'VisibilityHighMiles',
 'VisibilityAvgMiles',
 'VisibilityLowMiles',
 'WindHighMPH',
 'WindAvgMPH',
 'WindGustMPH',
 'PrecipitationSumInches']

In [449]:
X_train, X_test, y_train, y_test = train_test_split(df_austin[features], df_austin[target], random_state=34) 
forest = RandomForestClassifier(random_state=42, n_estimators=100, max_depth=7)
forest.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

Правильность на обучающем наборе: 0.941
Правильность на тестовом наборе: 0.885


In [406]:
X_train, X_test, y_train, y_test = train_test_split(df_austin[features], df_austin[target], random_state=34) 
tree_class = tree.DecisionTreeClassifier(
    random_state=0, max_depth=6, criterion='entropy', max_features=int(len(features)**0.5)
)
bag = BaggingClassifier(tree_class, random_state=42, n_estimators=150)
bag.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(bag.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(bag.score(X_test, y_test)))

Правильность на обучающем наборе: 0.923
Правильность на тестовом наборе: 0.885


In [450]:
X_train, X_test, y_train, y_test = train_test_split(df_austin[features], df_austin[target], random_state=34) 
tree_class = tree.DecisionTreeClassifier(
    random_state=0, max_depth=5, criterion='entropy', max_features=len(features)-2,
)
tree_class.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree_class.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree_class.score(X_test, y_test)))

Правильность на обучающем наборе: 0.887
Правильность на тестовом наборе: 0.876


In [241]:
for i in range(1, len(features)):
    X_train, X_test, y_train, y_test = train_test_split(df_austin[features], df_austin[target], random_state=42) 
    tree_class = tree.DecisionTreeClassifier(random_state=0, max_depth=5, criterion='entropy', max_features=i)
    tree_class.fit(X_train, y_train)
    # print("Правильность на обучающем наборе: {:.3f}".format(tree_class.score(X_train, y_train)))
    print("Правильность на тестовом наборе: {:.3f}".format(tree_class.score(X_test, y_test)))

Правильность на тестовом наборе: 0.736
Правильность на тестовом наборе: 0.706
Правильность на тестовом наборе: 0.812
Правильность на тестовом наборе: 0.773
Правильность на тестовом наборе: 0.788
Правильность на тестовом наборе: 0.748
Правильность на тестовом наборе: 0.833
Правильность на тестовом наборе: 0.736
Правильность на тестовом наборе: 0.827
Правильность на тестовом наборе: 0.818
Правильность на тестовом наборе: 0.830
Правильность на тестовом наборе: 0.748
Правильность на тестовом наборе: 0.809
Правильность на тестовом наборе: 0.830
Правильность на тестовом наборе: 0.806
Правильность на тестовом наборе: 0.830
Правильность на тестовом наборе: 0.842
Правильность на тестовом наборе: 0.824


Правильность на обучающем наборе: 0.846
Правильность на тестовом наборе: 0.773


In [372]:
tree.DecisionTreeClassifier?

In [ ]:
BaggingClassifier?